In [0]:
import re
import math
import tweepy
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

In [0]:
#Required for connection setup with twitter
ckey="A7uYpq8DnzOUiDtcNvyF5OM4D"
csecret="frAs2PXXNE7hbeSEjha8d6WpemZc62iENCLByTGLFlMUbMAs3O"
atoken="3190144129-PTVsgmcsJWJe6pgIl4VsXCVqz6AH7VqdUl6JhaW"
asecret="sExlOmhtm3sHNxYVWNYPdC92a8GuDi6C9VAZhJncAhARQ"

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)
api = tweepy.API(auth)

In [3]:
searchTerm = input("Enter keyword/hashtag you want to search :")
noOfTerms = int(input("Numper of tweets you want to analyze :"))

demoDF = pd.DataFrame(columns=['Date','Tweets'])

date = datetime.datetime.today()

#We can fetch only past 7 days's data
for i in range(8): 
    temp = str(date)
    tweets = tweepy.Cursor(api.search,q=searchTerm,lang="en",until=temp.split()[0]).items(noOfTerms)
    
    for tweet in tweets:
        demoDF = demoDF.append(pd.Series([tweet.created_at.date(),tweet.text], index=demoDF.columns),ignore_index=True)
        
    date += datetime.timedelta(days=-1)

Enter keyword/hashtag you want to search :AAPL
Numper of tweets you want to analyze :300


In [4]:
demoDF

,Date,Tweets
0,2020-04-23,RT @theAlgorithm__: $SPY $SPX $TSLA $AAPL $AMZ...
1,2020-04-23,$HNRG AFTERHOURS GIFT. Look how thin it is to ...
2,2020-04-23,$SPY $DIA $IWM $XHB $XRT $QQQ $SOXX\n\n$AAPL $...
3,2020-04-23,Shop Apple iPhone SE 2020 - Price Compare and ...
4,2020-04-23,$AAPL May 1 300 C @ 1.66 for 70% this morning....
...,...,...
2369,2020-04-16,Took some $AAPL long AH.
2370,2020-04-16,RT @scotwingo: The &gt; $1T horse-race is gett...
2371,2020-04-16,Quicker return to business as normal? Market s...
2372,2020-04-16,"$AAPL settled +0.8% to $286.69, and that made ..."


**Now we've to clean out tweets by removing unwanted texts like urls,mentions etc.....**

In [5]:
i=0
for tweet in demoDF.Tweets:
    tweet = tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    tweet = re.sub('@[^\s]+',' ',tweet)
    tweet = re.sub('[\s]+', ' ', tweet)
    tweet = re.sub(r'#([^\s]+)',' ', tweet)
    tweet = re.sub('at_user',' ',tweet)
    tweet = tweet.strip('\'"')
    demoDF.at[i,'Tweets'] = tweet
    i+=1
    
# Removing duplicate tweets...   
demoDF.drop_duplicates(subset=['Date','Tweets'],keep='first',inplace=True)
demoDF

,Date,Tweets
0,2020-04-23,rt $spy $spx $tsla $aapl $amzn 4 hour cmf on s...
1,2020-04-23,$hnrg afterhours gift. look how thin it is to ...
2,2020-04-23,$spy $dia $iwm $xhb $xrt $qqq $soxx $aapl $msf...
3,2020-04-23,shop apple iphone se 2020 - price compare and ...
4,2020-04-23,$aapl may 1 300 c @ 1.66 for 70% this morning....
...,...,...
2368,2020-04-16,options trading pro cumulative p&amp;l for 4/1...
2369,2020-04-16,took some $aapl long ah.
2370,2020-04-16,rt the &gt; $1t horse-race is getting really i...
2371,2020-04-16,quicker return to business as normal? market s...


**Combining all the tweets Date wise**

In [0]:
combine = lambda tweet:",".join(tweet)
combined = demoDF.groupby("Date").agg({'Tweets':combine},inplace=True).reset_index()

In [7]:
combined.sort_values(by=['Date'],inplace=True)
combined

,Date,Tweets
0,2020-04-16,rt as we await plan to reopen the us he got ...
1,2020-04-17,"deirdre o’brien sells 9,137 shares of apple in..."
2,2020-04-18,has your love for your bed grown too? $aapl $a...
3,2020-04-19,$gdow once upon a time a massive ih&amp;s. fol...
4,2020-04-20,wti demand begins to grow as america begins dr...
5,2020-04-21,fed eased up creating a pocket of weakness in ...
6,2020-04-22,rt some of today most active stock options 4.2...
7,2020-04-23,rt $spy $spx $tsla $aapl $amzn 4 hour cmf on s...


In [8]:
combined['Tweets'][0]

'rt as we await plan to reopen the us   he got input from 200 of the country\'s most influential &amp; prominent ceos…,rt those clowns who predicted $spy going to $160 $aapl to $150 $tsla to $210 - will they admit being wrong, take responsib…,why not sign up for roughly $1 a day for my watch list of 10 volatile stocks to daytrade. i provide ticker, suppor… ,$ntes is crossing the high                 $nflx   ,rt a tripple whammy win with just $aapl and tsla talk about dialing things down to the highest odds giving huge gains and b…,😍😍😍😍anyone sign up for robinhood and follow my advice yet? $amzn $aapl $tsla $nio $bbby $amd $nvda $roku i am giv… ,$abt crossing the high                 $nflx   ,rt inter-tech sector rotation after hours. $aapl $msft $fb $googl inverse to previous runners $nflx $amzn. not sure i have see…,rt so futures ripping big time, so if this is real i want to give plays to you all... $v back thru 168 snaps to 179 $m…,rt good evening! $spx ramping higher after trump pr

**Using tweepy, we can extract only last 7 days's data which is not sufficient enough for model training**

**So that's why we've downloaded data from the internet**

In [27]:
#Importing Data
tweets = pd.read_csv(r"/content/drive/My Drive/Project/apple-tweets.csv",engine='python')
tweets

,Date,Tweets
0,6/15/2016,RT @kirillklip: Race for #renewable #energy #t...
1,6/15/2016,#APPLE TRIES to Limit #Google Incursions Onto ...
2,6/15/2016,#APPLE TRIES to Limit #Google Incursions Onto ...
3,6/15/2016,What calls do I sell on $twtr\n\n$spy $aapl $g...
4,6/15/2016,Good for $AAPL pay? https://t.co/6EuitFi8Au
...,...,...
166626,4/2/2016,�And in a world where you get a huge cheer fro...
166627,4/2/2016,Why Intel Is Supporting #Apple's iPhone 7. Rea...
166628,4/2/2016,Why Intel Is Supporting #Apple's iPhone 7. Rea...
166629,4/2/2016,Here's why #Apple is flying a pirate flag to c...


**Now we've to perform all the data preprocessing again on this new dataset**

In [28]:
i=0
for tweet in tweets.Tweets:
    tweet = tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    tweet = re.sub('@[^\s]+',' ',tweet)
    tweet = re.sub('[\s]+', ' ', tweet)
    tweet = re.sub(r'#([^\s]+)',' ', tweet)
    tweet = re.sub('at_user',' ',tweet)
    tweet = tweet.strip('\'"')
    tweets.at[i,'Tweets'] = tweet
    i+=1
    
    
tweets.drop_duplicates(subset=['Date','Tweets'],keep='first',inplace=True)
tweets

,Date,Tweets
0,6/15/2016,rt race for charges market $lit ...
1,6/15/2016,tries to limit incursions onto its devices...
3,6/15/2016,what calls do i sell on $twtr $spy $aapl $goog...
4,6/15/2016,good for $aapl pay?
5,6/15/2016,apple may again see a super cycle akin to the ...
...,...,...
166625,4/2/2016,iphone growth will be there with se &amp; 7. ...
166626,4/2/2016,�and in a world where you get a huge cheer fro...
166627,4/2/2016,why intel is supporting iphone 7. read more:...
166629,4/2/2016,here's why is flying a pirate flag to celebr...


**Combining all the tweets date wise**

In [29]:
combine = lambda tweet:",".join(tweet)
grouped = tweets.groupby("Date").agg({'Tweets':combine},inplace=True).reset_index()
grouped.sort_values(by=['Date'],inplace=True)
grouped

,Date,Tweets
0,4/10/2016,"siri, kobe bryant &amp; michael b. jordan feat..."
1,4/11/2016,looks like there is a lot of spam going on in ...
2,4/12/2016,rt great vcs care about market share about as ...
3,4/13/2016,apple vs fang $aapl +17.5% vs +3.8% avg sinc...
4,4/14/2016,"rt $tsla hft algos triggered sell in sigma-x, ..."
...,...,...
64,6/3/2016,"$aapl inside week...,rt the way we interact wi..."
65,6/4/2016,should buy uber - business insider. read mor...
66,6/5/2016,"$aapl,rt volatility can be your friend. ..."
67,6/6/2016,"$spy $aapl next picks to be issued.,rt $aapl t..."


**We change date format for later use**

In [30]:
for i in range(len(grouped)):
    date = grouped.Date[i]
    temp = datetime.datetime.strptime(date, "%m/%d/%Y").strftime("%Y-%m-%d")
    grouped.at[i,'Date'] = temp

    
grouped

,Date,Tweets
0,2016-04-10,"siri, kobe bryant &amp; michael b. jordan feat..."
1,2016-04-11,looks like there is a lot of spam going on in ...
2,2016-04-12,rt great vcs care about market share about as ...
3,2016-04-13,apple vs fang $aapl +17.5% vs +3.8% avg sinc...
4,2016-04-14,"rt $tsla hft algos triggered sell in sigma-x, ..."
...,...,...
64,2016-06-03,"$aapl inside week...,rt the way we interact wi..."
65,2016-06-04,should buy uber - business insider. read mor...
66,2016-06-05,"$aapl,rt volatility can be your friend. ..."
67,2016-06-06,"$spy $aapl next picks to be issued.,rt $aapl t..."


**This dataset is not organised, So creating new dataframe with sorted data date wise**

In [31]:
tweets = pd.DataFrame(columns = ['Date','Tweets'])

date = datetime.datetime(2016,6,15)
for i in range(75):
    temp = str(date)
    temp = temp.split()[0]
    
    for j in range(len(grouped)):
        if temp == grouped.Date[j]:
            tweets = tweets.append(pd.Series([grouped.Date[j],grouped.Tweets[j]], index=tweets.columns),ignore_index=True)
            
    date += datetime.timedelta(days=-1)


tweets

,Date,Tweets
0,2016-06-15,rt race for charges market $lit ...
1,2016-06-14,"rt the 7 best new features of ios 10 $aapl ,ec..."
2,2016-06-07,&lt; review of this fantastic traders teachin...
3,2016-06-06,"$spy $aapl next picks to be issued.,rt $aapl t..."
4,2016-06-05,"$aapl,rt volatility can be your friend. ..."
...,...,...
64,2016-04-06,"breaking: apple introduces iphone 6, iphone 6 ..."
65,2016-04-05,"rt $txhd textmunication holdings, inc. issues ..."
66,2016-04-04,apple inc. $aapl shares sold by moody lynn &am...
67,2016-04-03,"as turns 40, do its best days lie ahead?. re..."


**importing stock price data downloaded from Yahoo Finance**

In [32]:
stocks_data = pd.read_csv(r"/content/drive/My Drive/Project/AAPL_share.csv",engine='python')
stocks_data = stocks_data.rename(columns={'Adj Close':'AdjClose'})
stocks_data

,Date,Open,High,Low,Close,AdjClose,Volume
0,2016-04-04,110.419998,112.190002,110.269997,111.120003,104.154839,37356200
1,2016-04-05,109.510002,110.730003,109.419998,109.809998,102.926964,26578700
2,2016-04-06,110.230003,110.980003,109.199997,110.959999,104.004860,26404100
3,2016-04-07,109.949997,110.419998,108.120003,108.540001,101.736557,31801900
4,2016-04-08,108.910004,109.769997,108.169998,108.660004,101.849045,23581700
5,2016-04-11,108.970001,110.610001,108.830002,109.019997,102.186470,29407500
6,2016-04-12,109.339996,110.500000,108.660004,110.440002,103.517471,27232300
7,2016-04-13,110.800003,112.339996,110.800003,112.040001,105.017174,33257300
8,2016-04-14,111.620003,112.389999,111.330002,112.099998,105.073410,25473900
9,2016-04-15,112.110001,112.300003,109.730003,109.849998,102.964447,46939000


**Merging two dataframes date wise into one named 'tweets'**

In [0]:
for i in range(len(tweets)):
    for j in range(len(stocks_data)):
        tweet_date = tweets.Date[i]
        stock_date = stocks_data.Date[j]
        
        if tweet_date == stock_date:
            tweets.at[i,'Price'] = stocks_data.AdjClose[j]

In [18]:
tweets

,Date,Tweets,Price
0,2016-06-15,rt race for charges market $lit ...,NaN
1,2016-06-14,"rt the 7 best new features of ios 10 $aapl ,ec...",91.907257
2,2016-06-07,&lt; review of this fantastic traders teachin...,93.387810
3,2016-06-06,"$spy $aapl next picks to be issued.,rt $aapl t...",93.010590
4,2016-06-05,"$aapl,rt volatility can be your friend. ...",NaN
...,...,...,...
64,2016-04-06,"breaking: apple introduces iphone 6, iphone 6 ...",104.004860
65,2016-04-05,"rt $txhd textmunication holdings, inc. issues ...",102.926964
66,2016-04-04,apple inc. $aapl shares sold by moody lynn &am...,104.154839
67,2016-04-03,"as turns 40, do its best days lie ahead?. re...",NaN


**Filling missing values of price with mean value**

In [34]:
summ = 0
total = 0

for price in tweets.Price:
    if not math.isnan(price):
        summ += price
        total +=1
    
mean = summ/total

for i in range(len(tweets)):
    if math.isnan(tweets.Price[i]):
        tweets.at[i,'Price'] = mean
        
tweets

,Date,Tweets,Price
0,2016-06-15,rt race for charges market $lit ...,94.314743
1,2016-06-14,"rt the 7 best new features of ios 10 $aapl ,ec...",91.907257
2,2016-06-07,&lt; review of this fantastic traders teachin...,93.387810
3,2016-06-06,"$spy $aapl next picks to be issued.,rt $aapl t...",93.010590
4,2016-06-05,"$aapl,rt volatility can be your friend. ...",94.314743
...,...,...,...
64,2016-04-06,"breaking: apple introduces iphone 6, iphone 6 ...",104.004860
65,2016-04-05,"rt $txhd textmunication holdings, inc. issues ...",102.926964
66,2016-04-04,apple inc. $aapl shares sold by moody lynn &am...,104.154839
67,2016-04-03,"as turns 40, do its best days lie ahead?. re...",94.314743


In [0]:
tweets['Price'] = tweets['Price'].apply(np.int64)

In [0]:
tweets_copy = pd.DataFrame(tweets)

**Applying sentiment analyzer (TextBlob)**

In [0]:
tweets['Polarity'] = ''

for i in range(len(tweets)):
    tweet = tweets.at[i,"Tweets"]
    sentiments = TextBlob(tweet)
    tweets.at[i,'Polarity'] = sentiments.sentiment.polarity

tweets = tweets.drop(['Tweets'],axis=1)

In [38]:
tweets

,Date,Price,Polarity
0,2016-06-15,94,0.197691
1,2016-06-14,91,0.130966
2,2016-06-07,93,0.261601
3,2016-06-06,93,0.171605
4,2016-06-05,94,0.183928
...,...,...,...
64,2016-04-06,104,0.193722
65,2016-04-05,102,0.1271
66,2016-04-04,104,0.165965
67,2016-04-03,94,0.183381


**we have also used NLTK library**

In [0]:
'''
nltk.download('vader_lexicon')
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata

analyzer = SentimentIntensityAnalyzer()

i=0
for tweet in tweets.Tweets:
    sentence = unicodedata.normalize('NFKD', tweet)
    sentence_sentiment = analyzer.polarity_scores(sentence)
    tweets.at[i,'Comp'] = sentence_sentiment['compound']
    tweets.at[i,'Positive'] = sentence_sentiment['pos']
    tweets.at[i,'Negative'] = sentence_sentiment['neg']
    tweets.at[i,'Neutral'] = sentence_sentiment['neu']
    i+=1

'''

In [39]:
col_list = ['Date','Price','Comp','Positive','Negative','Neutral']
df = pd.read_csv('/content/drive/My Drive/Project/finalDemoDataset.csv',usecols=col_list)
df

,Date,Price,Comp,Positive,Negative,Neutral
0,2016-06-15,94,0.9997,0.079,0.033,0.887
1,2016-06-14,91,0.9996,0.093,0.045,0.862
2,2016-06-07,93,0.9996,0.072,0.028,0.900
3,2016-06-06,93,1.0000,0.082,0.047,0.871
4,2016-06-05,94,0.9999,0.091,0.047,0.862
...,...,...,...,...,...,...
64,2016-04-06,104,1.0000,0.105,0.039,0.857
65,2016-04-05,102,1.0000,0.092,0.049,0.859
66,2016-04-04,104,1.0000,0.104,0.040,0.856
67,2016-04-03,94,0.9999,0.089,0.047,0.864


**used to summarize the strength of the linear relationship between two data samples**

**+ve value : variables change in the same direction**

**-ve value : variables change in different directions**

In [41]:
from scipy.stats import pearsonr
corr, _ = pearsonr(tweets['Price'], tweets['Polarity'])
print('Pearsons correlation between stock prices and twitter sentiments: %.3f' % corr)

Pearsons correlation between stock prices and twitter sentiments: 0.368
